# AVITO PROJECT

## Team Member

* Sandikha Rahardi - РИМ 130908

## Load Data

### Import Package

In [241]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')


from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

### Set Dataframe

In [242]:
dataset_filename = "assets/avito_dataset.xlsx"
main_df = pd.read_excel(dataset_filename, sheet_name="Sheet1")

## Data Cleaning

### Important Column

In [243]:
# Define important column to predict the price
important_columns = [
    'Цена', 'Метро', 'Метро2', 'Метро. Время', 
    'Район', 'Продавец. Компания', 'Статус', 'Количество комнат',
    'Общая площадь, число', 'Этаж', 'Санузел', 
    'Балкон или лоджия', 'Окна', 'Ремонт', 
    'Тип дома', 'Этажей в доме', 'Площадь кухни, число'
]
print("Total original column : " , main_df.columns.__len__())
print("Total important column : " , important_columns.__len__())
main_df = main_df[important_columns]

Total original column :  71
Total important column :  17


In [244]:
print(main_df.shape)
main_df.isnull().sum() / main_df.shape[0] * 100

(19803, 17)


Цена                     0.000000
Метро                    0.489825
Метро2                  47.856385
Метро. Время            47.856385
Район                   52.310256
Продавец. Компания       0.000000
Статус                   0.000000
Количество комнат        0.000000
Общая площадь, число     0.000000
Этаж                     0.000000
Санузел                 40.301974
Балкон или лоджия       32.692016
Окна                    56.824724
Ремонт                  60.793819
Тип дома                13.750442
Этажей в доме            0.000000
Площадь кухни, число    19.491996
dtype: float64

### Clean the dataset

#### Remove duplicated dataset

In [245]:
print("Duplicated data : ", main_df.duplicated().sum())
main_df = main_df.drop_duplicates().reset_index(drop=True)
print("Duplicated data (now) : ", main_df.duplicated().sum())

Duplicated data :  276
Duplicated data (now) :  0


#### Clean about 'Метро'

In [246]:
main_df.dropna(subset=['Метро'], inplace=True)

def preprocess_metro_time(row):
    if(type(row) == str):
        row = row.replace("мин.","").strip()
        if(row.__contains__("от")):
            row = float(row.replace("от","").strip())
        elif(row.__contains__("до")):
            row = float(row.replace("до","").strip())
        elif(row.__contains__("–")):
            row = (int(row.split("–")[0]) + int(row.split("–")[1])) / 2
    return row

main_df['Метро. Время'] = main_df['Метро. Время'].apply(preprocess_metro_time)
mean_metro_time = round(pd.to_numeric(main_df['Метро. Время']).mean(), 1)
main_df['Метро. Время'].fillna(mean_metro_time, inplace=True)
main_df['Метро2'] = main_df['Метро2'].apply(lambda x: 0 if pd.isna(x) else 1)

#### Clean about 'Район'

In [247]:
main_df['Район'].fillna("Другой", inplace=True)

#### Clean about 'Продавец. Компания'

In [248]:
main_df['Продавец. Компания'] = main_df['Продавец. Компания'].apply(lambda x: 0 if str(x) == "Нет" else 1)

#### Clean about 'Количество комнат'

In [249]:
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: 0 if x == 'Студия' else x)

max_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').max()
mean_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').mean()

main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: max_room if x == 'многокомнатные' else x)
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: round(mean_room) if x == 'Своб. планировка' else x)

> Note: Студия does not have room ( 0 room )

#### Clean about 'Общая площадь, число'

In [250]:
main_df['Общая площадь, число'] = main_df['Общая площадь, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else float(x))

#### Clean about 'Этаж'

In [251]:
# Clean dataset 'Этаж' of 'Этажей в доме'
main_df['Этаж'] = main_df['Этаж'].apply(lambda x: int(x.split("из")[0].strip()))

#### Clean about 'Санузел'

In [252]:
main_df['Санузел'].fillna('Другой', inplace=True)

#### Clean about 'Балкон или лоджия'

In [253]:
main_df['Балкон или лоджия'].fillna('Другой', inplace=True)

#### Clean about 'Окна'

In [254]:
main_df['Окна'].fillna('Другой', inplace=True)

#### Clean about 'Ремонт'

In [255]:
main_df['Ремонт'].fillna('Другой', inplace=True)

#### Clean about 'Тип дома'

In [256]:
main_df['Тип дома'].fillna('Другой', inplace=True)

#### Clean about 'Площадь кухни, число'

In [257]:
main_df['Площадь кухни, число'] = main_df['Площадь кухни, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else float(x))
main_df['Площадь кухни, число'].fillna(main_df['Площадь кухни, число'].mode()[0], inplace=True)

### Split Dataset

In [258]:
main_df.head()

,Цена,Метро,Метро2,Метро. Время,Район,Продавец. Компания,Статус,Количество комнат,"Общая площадь, число",Этаж,Санузел,Балкон или лоджия,Окна,Ремонт,Тип дома,Этажей в доме,"Площадь кухни, число"
0,5100000,м. р-н Академический,0,22.7,Академический,0,Квартира,2.0,53.6,3,Раздельный,Лоджия,Другой,Косметический,Монолитный,5,12.8
1,4470000,м. Геологическая,1,31.0,Другой,0,Квартира,2.0,36.1,4,Совмещённый,Балкон,"На улицу, На солнечную сторону",Косметический,Панельный,5,5.0
2,8650000,м. Уральская,1,25.5,Другой,0,Квартира,3.0,59.0,11,Раздельный,Балкон,Во двор,Евро,Монолитный,25,23.0
3,12580000,м. р-н Академический,0,22.7,Академический,1,Квартира,2.0,97.8,9,Совмещённый,Балкон,"Во двор, На улицу",Другой,Монолитный,9,22.9
4,11340000,м. р-н Академический,0,22.7,Академический,1,Квартира,1.0,86.4,9,Совмещённый,Балкон,"Во двор, На улицу",Другой,Монолитный,9,27.8


In [259]:
# Split the data into training (80%), testing (15%), and validation (5%)
train_df, temp_df = train_test_split(main_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.25, random_state=42)
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(15544, 17)
(2914, 17)
(972, 17)


### Divided into Categorical and Numerical dataset

In [260]:
label = 'Цена'
numerical_columns = ['Метро. Время', 'Количество комнат', 'Общая площадь, число', 'Этаж', 'Этажей в доме', 'Площадь кухни, число']
categorical_columns = ['Метро', 'Метро2', 'Район', 'Продавец. Компания', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Тип дома']

train_num_df = train_df[numerical_columns]
train_cat_df = train_df[categorical_columns]
test_num_df = test_df[numerical_columns]
test_cat_df = test_df[categorical_columns]
val_num_df = val_df[numerical_columns]
val_cat_df = val_df[categorical_columns]

### Dummies Variables

In [261]:
preprocessing_categorical_columns = ['Метро', 'Район', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Тип дома']
main_df[preprocessing_categorical_columns].head()

,Метро,Район,Статус,Санузел,Балкон или лоджия,Окна,Ремонт,Тип дома
0,м. р-н Академический,Академический,Квартира,Раздельный,Лоджия,Другой,Косметический,Монолитный
1,м. Геологическая,Другой,Квартира,Совмещённый,Балкон,"На улицу, На солнечную сторону",Косметический,Панельный
2,м. Уральская,Другой,Квартира,Раздельный,Балкон,Во двор,Евро,Монолитный
3,м. р-н Академический,Академический,Квартира,Совмещённый,Балкон,"Во двор, На улицу",Другой,Монолитный
4,м. р-н Академический,Академический,Квартира,Совмещённый,Балкон,"Во двор, На улицу",Другой,Монолитный


In [262]:
preprocessing_categorical_columns = ['Метро', 'Район', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Тип дома']
all_dummies_variable = []
dict_dummies_variable = dict()

for cat_column in preprocessing_categorical_columns:
    all_data = []
    dummies_df = main_df[cat_column].value_counts().reset_index()[cat_column]
    
    for index_data in range(len(dummies_df)):
        data = dict()
        data["dummies"] = index_data
        data["value"] = dummies_df[index_data]
        all_data.append(data)
        main_df[cat_column] = main_df[cat_column].apply(lambda x: index_data if x == dummies_df[index_data] else x)
    
    dict_dummies_variable[cat_column] = all_data
    all_dummies_variable.append(dict_dummies_variable)
    
main_df[preprocessing_categorical_columns].head()

,Метро,Район,Статус,Санузел,Балкон или лоджия,Окна,Ремонт,Тип дома
0,0,1,0,2,0,0,1,0
1,4,0,0,1,2,6,1,3
2,5,0,0,2,2,1,2,0
3,0,1,0,1,2,2,0,0
4,0,1,0,1,2,2,0,0


# ========= TEST ========= #

In [204]:
new_train_df = train_df[numerical_columns + categorical_columns + [label]]
new_train_df[categorical_columns] = new_train_df[categorical_columns].apply(lambda x: pd.factorize(x)[0])
new_train_df[categorical_columns]

,Метро,Метро2,Район,Продавец. Компания,Статус,Санузел,Балкон или лоджия,Окна,Ремонт,Тип дома
11161,0,0,0,0,0,0,0,0,0,0
12530,1,0,1,1,0,0,1,1,0,1
15445,2,1,2,0,0,1,1,2,0,2
0,1,0,1,0,0,1,2,0,0,1
4104,3,1,2,1,0,0,2,3,0,1
...,...,...,...,...,...,...,...,...,...,...
11354,5,0,3,1,0,2,2,5,2,1
12036,2,1,2,1,0,2,2,0,2,1
5424,14,1,2,0,0,3,2,4,4,2
864,9,1,2,1,0,2,2,0,2,1


In [205]:
new_train_df

,Метро. Время,Количество комнат,"Общая площадь, число",Этаж,Этажей в доме,"Площадь кухни, число",Метро,Метро2,Район,Продавец. Компания,Статус,Санузел,Балкон или лоджия,Окна,Ремонт,Тип дома,Цена
11161,22.7,1.0,18.0,5,5,6.0,0,0,0,0,0,0,0,0,0,0,1950000
12530,22.7,1.0,38.0,9,10,10.0,1,0,1,1,0,0,1,1,0,1,4500000
15445,25.5,2.0,55.6,3,3,7.0,2,1,2,0,0,1,1,2,0,2,4900000
0,22.7,2.0,53.6,3,5,12.8,1,0,1,0,0,1,2,0,0,1,5100000
4104,18.0,1.0,36.4,6,16,12.0,3,1,2,1,0,0,2,3,0,1,4300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11354,22.7,2.0,58.3,22,24,11.8,5,0,3,1,0,2,2,5,2,1,7870000
12036,25.5,3.0,81.3,3,24,29.1,2,1,2,1,0,2,2,0,2,1,9590000
5424,31.0,3.0,84.7,4,12,15.3,14,1,2,0,0,3,2,4,4,2,13000000
864,25.5,1.0,51.0,15,30,22.6,9,1,2,1,0,2,2,0,2,1,13091000


In [206]:
X = new_train_df[categorical_columns + numerical_columns]
y = new_train_df[label]

In [207]:
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X, y)

DecisionTreeClassifier(max_depth=3)

In [208]:
X.head()

,Метро,Метро2,Район,Продавец. Компания,Статус,Санузел,Балкон или лоджия,Окна,Ремонт,Тип дома,Метро. Время,Количество комнат,"Общая площадь, число",Этаж,Этажей в доме,"Площадь кухни, число"
11161,0,0,0,0,0,0,0,0,0,0,22.7,1.0,18.0,5,5,6.0
12530,1,0,1,1,0,0,1,1,0,1,22.7,1.0,38.0,9,10,10.0
15445,2,1,2,0,0,1,1,2,0,2,25.5,2.0,55.6,3,3,7.0
0,1,0,1,0,0,1,2,0,0,1,22.7,2.0,53.6,3,5,12.8
4104,3,1,2,1,0,0,2,3,0,1,18.0,1.0,36.4,6,16,12.0


In [216]:
y[15445]

4900000

In [217]:
tree.predict([[2,1,2,0,0,1,1,2,0,2,25.5,2.0,55.6,3,3,7.0]])

/Users/kuldii/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([5500000])